In [1]:
import functools

import numpy as np
from collections import defaultdict
from scipy.special import expit

import pyspark.sql.functions as psf
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.functions import udf, pandas_udf
from pyspark.sql.functions import col
import pyspark.sql as pysql
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, MapType, ArrayType, FloatType, BooleanType
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
# from operator import add
# from functools import reduce

pd.set_option('display.max_columns', None)
spark.conf.set('spark.sql.session.timeZone', 'UTC')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
236,application_1653679151702_0238,pyspark,idle,Link,Link,gakumar,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
date = '2022-06-13'
hr = '08'
env = 'production'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Loop Detection

In [3]:
from collections import defaultdict
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import BooleanType, ArrayType, StringType, IntegerType, MapType, FloatType, StructType, StructField


# import networkx as nx


class Cycles:
    def __init__(self, graph):
        self._graph = graph
        self.nodes = self.get_nodes()
        self.nodeMap = {node: i for i, node in enumerate(self.nodes)}
        self.visited = ['NOT_VISITED' for _ in range(len(self.nodes))]  # Initialize all nodes unvisited
        self.stack = []  # Stack to keep track of visited nodes
        self.cycles = []

    def get_nodes(self):
        _nodes = set([])
        for k, v in self._graph.items():
            items = [k] + v
            for item in items:
                _nodes.add(item)
        return list(_nodes)

    def print_cycles(self):
        return self.cycles

    def add_cycle(self, v):
        cycle = list()
        cycle.append(self.nodes[self.stack[-1]])
        i = 1
        while cycle[-1] != v and i < len(self.stack):
            i += 1
            cycle.append(self.nodes[self.stack[-i]])

        self.cycles.append(cycle)

    def dfs(self):
        curr = self.stack[-1]
        if self.nodes[curr] in self._graph:
            for neighbour in self._graph[self.nodes[curr]]:
                to = self.nodeMap[neighbour]
                if self.visited[to] == 'ON_STACK':
                    self.add_cycle(to)
                elif self.visited[to] == 'NOT_VISITED':
                    self.stack.append(to)
                    self.visited[to] = 'ON_STACK'
                    self.dfs()

        self.visited[curr] = 'DONE'
        self.stack.pop()

    def find_cycles(self):
        for i, node in enumerate(self.nodes):
            if self.visited[i] == 'NOT_VISITED':
                self.stack = []
                self.stack.append(i)
                self.visited[i] = 'ON_STACK'
                self.dfs()

        return self.print_cycles()


def get_site_graph(data_frame):
    df_site_adj_list = data_frame.select('siteId', 'src', 'dst') \
        .groupby('siteId', 'src') \
        .agg(F.collect_set(F.col('dst')).alias('dst'))
    df_site_adj_list = df_site_adj_list.groupby("siteId") \
        .agg(F.map_from_arrays(F.collect_list("src"), F.collect_list("dst")).alias("graph"))

    return df_site_adj_list


def _is_cycle(graph, v, done, stack):
    done[v] = True
    stack[v] = True

    for neighbour in graph[v]:
        if neighbour in graph:
            if not done[neighbour]:
                if _is_cycle(graph, neighbour, done, stack):
                    print(f'Part of cycle: {neighbour}')
                    return True
            elif stack[neighbour]:
                print(f'Last neighbour found on stack: {neighbour}')
                return True

    stack[v] = False
    return False


def is_cycle(graph):
    src_vertices = graph.keys()
    done = {k: False for k in src_vertices}
    stack = {k: False for k in src_vertices}

    for node in src_vertices:
        if not done[node]:
            if _is_cycle(graph, node, done, stack):
                return True
    return False


def find_cycles(graph):
    cycles = Cycles(graph)
    return cycles.find_cycles()


# def find_cycles_nx(graph):
#     G = nx.DiGraph(graph)
#     return list(nx.simple_cycles(G))
#
# udf_find_cycles_nx = F.udf(find_cycles_nx, ArrayType(ArrayType(StringType())))


# Added for site coverage caluclation
def reverse_graph(graph):
    k = graph.keys()
    v = graph.values()
    
    rev_graph = defaultdict(list)
    
    for key, value in zip(k, v):
        for val in value:
            rev_graph[val].append(key)
            
    return rev_graph


def _device_tag(graph, v, done):
    done[v] = True

    for neighbour in graph[v]:
        if neighbour in graph:
            if not done[neighbour]:
                _device_tag(graph, neighbour, done)
        else:
            done[neighbour] = True

    return None


def get_path(x):
    init_graph = x[0]
    df_sel_nodes = x[1]
    graph = reverse_graph(init_graph)
    src_vertices = graph.keys()
    
    target_vertices = []
    for values in graph.values():
        for val in values:
            target_vertices.append(val)
            
    vertices = set(src_vertices) | set(target_vertices)
    
    done = {k: False for k in vertices}

    source_macs = set([node.split("__")[0] for node in src_vertices])
    target_macs = set([node.split("__")[0] for node in init_graph.keys()])
    
    starting_nodes = source_macs - target_macs
    
    starting_vertices = [node for node in src_vertices if (node.split("__")[0] in starting_nodes)&(node.split("__")[0] in df_sel_nodes)]
    
    for node in starting_vertices:
        if not done[node]:
            _device_tag(graph, node, done)
    return done


udf_visit_count = F.udf(lambda x: len(set([device.split("__")[0] for device, visited in x.items() if visited == True])), IntegerType())

udf_node_count = F.udf(lambda x: len(set([device.split("__")[0] for device in x.keys()])), IntegerType())

udf_coverage = F.udf(lambda x: float(x[0])/float(x[1]), FloatType())
                                        
udf_get_path = F.udf(get_path, MapType(StringType(), BooleanType()))

udf_find_cycles = F.udf(find_cycles, ArrayType(ArrayType(StringType())))

udf_is_cycle = F.udf(is_cycle, BooleanType())

udf_array_size = F.udf(lambda a: [len(e) for e in a], ArrayType(IntegerType()))

udf_extract_mac = F.udf(lambda a: [s.split('__')[0] for lst in a for s in lst if '__' in s], ArrayType(StringType()))

uplink_match_schema = StructType([
    StructField("match", IntegerType()),  
    StructField("curr_mismatch", IntegerType()),
    StructField("pred_mismatch", IntegerType())
])

@F.udf(returnType=uplink_match_schema)
def uplink_match(uplink_curr, uplink_pred):
    if uplink_curr is None:
        if uplink_pred is None:
            return 0, 0, 0
        else:
            return 0, 0, len(uplink_pred)
    elif len(uplink_curr) == 0:
        if uplink_pred is None:
            return 0, 0, 0
        else:
            return 0, 0, len(uplink_pred)
    elif len(uplink_curr) > 0:
        if uplink_pred is None:
            return 0, len(uplink_curr), 0
        else:
            match = len(set(uplink_curr) & set(uplink_pred))
            curr_mismatch = len(uplink_curr) - match
            pred_mismatch = len(uplink_pred) - match
            return match, curr_mismatch, pred_mismatch



def compute(edges, nodes):

#     df_oc = consistency_check(df_oc, df_new_uplink).persist()

#     old_preds_switches = df_oc.filter(F.size("uplink_combined")>0).count()
#     any_match = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)).count()
#     total_match = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")==0)&(F.col("pred_mismatch")==0)).count()
#     less_uplink_pred = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")>0)&(F.col("pred_mismatch")==0)).count()
#     more_uplink_pred = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")==0)&(F.col("pred_mismatch")>0)).count()
#     new_uplink_pred = df_oc.filter((~F.col("uplink_new").isNull()) & (F.size("uplink_combined")==0)).count()

#     no_uplink_pred = df_oc.filter((F.col("uplink_new").isNull()) & (F.size("uplink_combined")>0)).count()
#     diff_uplink_pred = df_oc.filter((~F.col("uplink_new").isNull()) & (F.size("uplink_combined")>0) & (F.col("match")==0)).count()
    
#     # Logging Consistency Metrics
#     print('Total switches with existing predictions : {}'.format(old_preds_switches))

#     print('Among existing, any match : {}'.format(any_match))

#     print('Among existing, total match : {}'.format(total_match))

#     print('Among existing, less uplinks predicted : {}'.format(less_uplink_pred))

#     print('Among existing, more uplinks predicted : {}'.format(more_uplink_pred))

#     print('New uplinks predicted : {}'.format(new_uplink_pred))
    
#     print('Among existing, no uplink predicted: {}'.format(no_uplink_pred))
    
#     print('Among existing, different uplink predicted: {}'.format(diff_uplink_pred))

#     # Importing from old predictions where new predictions are unavailable
#     updated_edges = mix_predictions(df_oc, edges).persist()

    active_edges, candidate_node = process_data(edges, nodes)

    active_edges_count = active_edges.count()
    print('Total active_edges_count : {}'.format(active_edges_count))

    site_graph_cycle_true = get_sites_with_cycles(active_edges).persist()

    sites_with_cycles = site_graph_cycle_true.count()
    print('Sites with cycles in device collection(relType==uplink) = {}'.format(sites_with_cycles))

    # For site coverage
    df_cycle_cov = site_graph_cycle_true.join(candidate_node, 
                                    (site_graph_cycle_true.siteId == candidate_node.siteId_copy), 
                                    how="left").drop("siteId_copy")
    df_cov_stats = df_cycle_cov.withColumn("nodes_covered", 
                    udf_get_path(F.struct("graph", "mac_list"))) \
        .withColumn("visited_count", udf_visit_count(F.col("nodes_covered"))) \
        .withColumn("node_count", udf_node_count(F.col("nodes_covered"))) \
        .withColumn("coverage", udf_coverage(F.struct(F.col("visited_count"), F.col("node_count")))) \
        .select(*["siteId", "visited_count", "node_count", "coverage"]).persist()

    top_loop_count = df_cov_stats.filter(F.col("visited_count") == 0).count()
    print('Total sites with top level loops {}'.format(top_loop_count))
    
    stats = df_cov_stats.agg({"visited_count":"sum", "node_count": "sum", "coverage":"mean"}).collect()
    print('Coverage in sites with loops {}'.format(stats[0][0]))

    print('Devices left out in sites with loops {}'.format(stats[0][2] - stats[0][1]))
    
    df_site_graph_all_cycles = site_graph_cycle_true.withColumn('cycles', udf_find_cycles(F.col('graph')))
    # df_site_graph_all_cycles_nx = site_graph_cycle_true.withColumn('cycles', udf_find_cycles_nx(F.col('graph')))
    # df_graph_cycles_nx_pd = df_site_graph_all_cycles_nx.select('siteId', 'cycles').toPandas()

    df_graph_cycles = df_site_graph_all_cycles \
        .select('siteId', 'cycles'). \
        withColumn('cycle_length', udf_array_size(F.col('cycles'))). \
        withColumn('num_cycles', F.size(F.col('cycle_length')))

    total_cycles = df_graph_cycles.select(
        F.sum(F.col('num_cycles')).alias('total_cycles')
    ).rdd.map(lambda x: x.asDict()).first()
    if total_cycles:
        print('new_model_total_cycles: ', total_cycles)

    df_device_macs = df_site_graph_all_cycles.select('siteId', 'cycles') \
        .withColumn('macs', udf_extract_mac(F.col('cycles')))
    df_macs = df_device_macs.select(F.explode(F.col('macs')).alias('mac')).distinct()

    total_device_count = df_macs.count()
    print('Total device involved in loops {}'.format(total_device_count))

    df_combined_nodes = df_macs.join(nodes, df_macs.mac == nodes.mac)

    device_types = df_combined_nodes.groupBy('deviceType') \
        .count().rdd.map(lambda x: x.asDict()).collect()
    device_types = {r.get('deviceType'): r.get('count') for r in device_types}
    if device_types:
        print('Total device_types {}'.format(device_types))
    
    return df_cov_stats

def process_data(df_edges, df_nodes):
    """
    * Filter out deviceType=='ap'
    * create src, dst columns with mapping
        src-->(source,sourcePort)
        dst-->(target,targetPort)
    """
    print('Pre-processing node and edge data')
    df_edges = df_edges.withColumn('isExpired', F.when(F.isnull(F.col('expiredAt')), 'False').otherwise('True'))
    df_active_edges = df_edges.filter(F.col('isExpired') == F.lit(False))
    df_active_edges = df_active_edges.filter(F.col('relType') == 'uplink')

    df_nodes = df_nodes.withColumn('isExpired', F.when(F.isnull(F.col('expiredAt')), 'False').otherwise('True'))
    df_active_nodes = df_nodes.filter(F.col('isExpired') == F.lit(False))
    df_active_nodes = df_active_nodes.withColumnRenamed("siteId", "siteId_copy")

    # For site coverage
    candidate_node = df_active_nodes.filter(F.col("deviceType").isin(['switch', 'gateway', 'third'])) \
        .where(F.col("model") != 'SSR') \
        .where(~((F.isnull(F.col("mac")))|(F.col("mac")==''))) \
        .select(*["siteId_copy", "mac", "deviceType"]).distinct() \

    candidate_node = candidate_node.groupby(F.col("siteId_copy")).agg(F.collect_list(F.col("mac")).alias("mac_list"))

    df_join = df_active_edges.join(df_active_nodes, [df_active_edges.source == df_active_nodes.mac,
                                                     df_active_edges.siteId == df_active_nodes.siteId_copy]) \
        .drop('siteId_copy')

    df_active_nodes = df_active_nodes.withColumn('mac_target', df_active_nodes.mac) \
        .withColumn('deviceType_target', df_active_nodes.deviceType)

    df_join = df_join.join(df_active_nodes.select('mac_target', 'deviceType_target', 'siteId_copy'),
                           [df_join.target == df_active_nodes.mac_target,
                            df_join.siteId == df_active_nodes.siteId_copy])

    df_active_edges_filtered = df_join.filter((F.col('deviceType') != 'ap') & (F.col('deviceType_target') != 'ap')) \
        .select('siteId', 'source', 'target', 'sourcePort', 'targetPort', 'sourceVendor', 'targetVendor',
                'deviceType',
                'deviceType_target', 'relType')

    df_active_edges_filtered = df_active_edges_filtered.withColumn('src', F.concat_ws("__", F.col('source'),
                                                                                      F.col('sourcePort'))) \
        .withColumn('dst', F.concat_ws("__", F.col('target'), F.col('targetPort')))

    return df_active_edges_filtered, candidate_node

def get_sites_with_cycles(data_frame):
    print('Building graph in get_sites_with_cycles')
    df_site_graph = get_site_graph(data_frame)
    df_site_graph_cycle = df_site_graph.withColumn('cycle_detected', udf_is_cycle(F.col('graph')))
    return df_site_graph_cycle.filter(F.col('cycle_detected') == F.lit(True))

def consistency_check(df_oc, df_new):
    df_new = df_new.filter((F.col("uplink_value")==1) & (F.col("n_ind")==1)).groupby("site_id", "switch_id").agg(
        F.collect_list("name").alias("uplink_new")
    )

    rank_window = Window().partitionBy(F.col('switch_id')).orderBy(F.col("when").desc())
    df_oc = df_oc.withColumn("rank", F.rank().over(rank_window)).filter(F.col("rank")==1)
    df_oc = df_oc.select("site_id", "switch_id", "uplink_combined")

    df_oc = df_oc.join(df_new, ["site_id", "switch_id"], how="outer")
    df_oc = df_oc.withColumn("match", uplink_match("uplink_combined", "uplink_new")) \
        .select('site_id', 'switch_id', 'uplink_combined', 'uplink_new', F.col('match.*'))

    return df_oc

def mix_predictions(df, df_edges):
    df = df.withColumn("mixed_uplink", F.when(F.col("uplink_new").isNull(), 
        F.col("uplink_combined")).otherwise(F.col("uplink_new")))

    df = df.select('site_id', 'switch_id', F.explode('mixed_uplink').alias("name")) \
        .withColumn("uplink_value", F.lit(1)).persist()

    df = df.withColumnRenamed("site_id", "siteId") \
        .withColumnRenamed("switch_id", "source") \
        .withColumnRenamed("name", "sourcePort")

    df_edges = df_edges.join(df, ["siteId", "source", "sourcePort"], how="left")
    df_edges = df_edges.drop("relType")
    df_edges = df_edges.withColumn("relType", F.when(F.col("uplink_value") == 1, 'uplink').otherwise(''))
    return df_edges

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
s3_path = 's3://mist-aggregated-stats-'+env+'/aggregated-stats/graph/snapshots/device-edges/dt='+date+'/hr='+hr+'/*'
node_s3_path = 's3://mist-aggregated-stats-'+env+'/aggregated-stats/graph/snapshots/device-nodes/dt='+date+'/hr='+hr+'/*'
oc_stats_s3_path = 's3://mist-secorapp-'+env+'/oc-stats-analytics/oc-stats-analytics-'+env+'/dt='+date+'/hr='+hr+'/*'

print(s3_path)
print(node_s3_path)
print(oc_stats_s3_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-edges/dt=2022-06-13/hr=08/*
s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-nodes/dt=2022-06-13/hr=08/*
s3://mist-secorapp-production/oc-stats-analytics/oc-stats-analytics-production/dt=2022-06-13/hr=08/*

In [5]:
print('reading from s3')
df_edge = spark.read.parquet(s3_path)
df_node = spark.read.parquet(node_s3_path)
df_oc_stat = spark.read.parquet(oc_stats_s3_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

reading from s3

In [8]:
# Loading df_uplink
uplink_pred_s3_path = "s3://mist-aggregated-stats-"+env+"/aggregated-stats/switch_uplink_model_v2/dt="+date+'/hr='+hr+'/last_3_hour/*'
print(uplink_pred_s3_path)
df_uplink = spark.read.parquet(uplink_pred_s3_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-aggregated-stats-production/aggregated-stats/switch_uplink_model_v2/dt=2022-06-13/hr=08/last_3_hour/*

In [9]:
old_sites = compute(df_edge, df_node)
old_sites = old_sites.filter(F.col("visited_count") == 0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pre-processing node and edge data
Total active_edges_count : 137596
Building graph in get_sites_with_cycles
Sites with cycles in device collection(relType==uplink) = 80
Total sites with top level loops 62
Coverage in sites with loops 0.035086204297840595
Devices left out in sites with loops 6056
new_model_total_cycles:  {'total_cycles': 112}
Total device involved in loops 163
Total device_types {'gateway': 13, 'third': 2, 'switch': 148}

In [10]:
old_site_list = set(
    old_sites.select('siteId').toPandas()['siteId']
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
old_site_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'7cf7111a-b9e9-48df-86ef-230fae9c5a58', '10448f99-05ff-423f-b48e-696413d4c019', '72706575-6eec-42c2-80d8-5ec2a27d232b', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', '09df1500-f1b3-4501-a805-7af21a1fb90f', '7933e726-0927-440f-b0f6-0dffeb54b8a2', '4d6abae0-f9eb-40fe-9db2-ccb630d1261b', '462562db-5118-4581-bfc7-0fc9d86eafc6', '785bd2d2-fb41-4c0d-a049-40fe0983a159', '7e9f985d-dfc3-4f23-bf71-33130bd7223f', '2c355e9f-8cdd-4aeb-811e-26907afb3a46', '77a8fec7-dce8-4486-843f-14ec33a99833', 'c6987f0f-c68f-43d9-993e-e9c1d63fc4d7', '54f41c7f-42c2-483c-a073-797964bfffed', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '88a9ac0f-f166-4a7a-aeff-e3b0f37c0844', '6647863e-ec58-4390-9c65-a68dcf270d24', '9c103626-af9d-41f4-ac8c-254cb61863f6', 'b065a63a-f500-428a-80bf-9bac6301317e', '3604f709-045b-459a-b8df-45a399ab56e4', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', 'd72db198-cb28-48d6-b931-dfe5782e8fe6',

## Check Merge with old new top loops

In [17]:
# date = '2022-05-29'
# hr = '15'
# Count = 21 new sites with loops
sites1 = {'0e9177ea-e28f-4741-9595-76ae7b88f755', '10448f99-05ff-423f-b48e-696413d4c019', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '9c103626-af9d-41f4-ac8c-254cb61863f6', 'f2860e64-78af-4708-9b64-a5b1299bf952', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', 'cd70c572-6191-41f7-9d54-885e8121619c', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', '10811140-1428-4592-88e5-64ca908a08b2', 'becdf2ec-cd2a-44e3-b9bb-bba05a2c39e2', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '898b809b-2b46-443d-babb-c374ce638b94', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}

# date = '2022-05-30'
# hr = '03'
# Count = 14 new sites with loops
sites2 = {'766c8be2-dc5c-4e3a-b88d-a14ce6143492', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', '898b809b-2b46-443d-babb-c374ce638b94', '06de2a1d-e72f-4d10-8cb4-34286e4a859e', '10811140-1428-4592-88e5-64ca908a08b2', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-30'
# hr = '12'
# Count = 18 new sites with loops
sites3 = {'10448f99-05ff-423f-b48e-696413d4c019', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', 'f62e5757-fc31-4442-bf01-eb5f89d27b6c', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', '898b809b-2b46-443d-babb-c374ce638b94', 'e3f9b45c-462c-471c-83be-503534e9f301', '10811140-1428-4592-88e5-64ca908a08b2', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}

# date = '2022-05-30'
# hr = '07'
# Count = 17 new sites with loops
sites4 = {'10448f99-05ff-423f-b48e-696413d4c019', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', '99a28ff4-3d06-4762-ae04-9b362307cd12', '2db0c421-8401-43bf-9a5c-0d901628a5e2', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '10811140-1428-4592-88e5-64ca908a08b2', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-25'
# hr = '10'
# Count = 18 new sites with loops
sites5 = {'7cf7111a-b9e9-48df-86ef-230fae9c5a58', '0e9177ea-e28f-4741-9595-76ae7b88f755', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', '10811140-1428-4592-88e5-64ca908a08b2', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-27'
# hr = '17'
# Count = 18 new sites with loops
sites6 = {'10448f99-05ff-423f-b48e-696413d4c019', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'bd6ca967-f622-40e9-b75a-bbdee6330e15', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '03baf34e-0186-4037-b8dc-88c80649532d', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}

sites_check = sites1 | sites2 | sites3 | sites4 | sites5 | sites6 
print(len(sites_check))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

31

In [18]:
len(sites_check & old_site_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22

In [19]:
print(sites_check & old_site_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'7cf7111a-b9e9-48df-86ef-230fae9c5a58', '10448f99-05ff-423f-b48e-696413d4c019', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '9c103626-af9d-41f4-ac8c-254cb61863f6', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', 'cd70c572-6191-41f7-9d54-885e8121619c', '03baf34e-0186-4037-b8dc-88c80649532d', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '898b809b-2b46-443d-babb-c374ce638b94', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', '06de2a1d-e72f-4d10-8cb4-34286e4a859e', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

In [20]:
for i in sites_check & old_site_list:
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7cf7111a-b9e9-48df-86ef-230fae9c5a58
10448f99-05ff-423f-b48e-696413d4c019
04d8a2d8-4b9c-4314-b1fb-7cec586f437d
d7e0e251-82a9-4a84-9622-844ca4fd1e4b
60a31a89-4a68-4a46-a58e-0a8cf866ef7a
9c103626-af9d-41f4-ac8c-254cb61863f6
128aa802-6601-4197-bb38-c9c2ae2d3994
f8a3c86b-6472-488e-a6c9-ee42d70a6f44
d08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8
0bc1ffb9-7815-4a4d-ba59-f674dd057d67
cd70c572-6191-41f7-9d54-885e8121619c
03baf34e-0186-4037-b8dc-88c80649532d
2db0c421-8401-43bf-9a5c-0d901628a5e2
99a28ff4-3d06-4762-ae04-9b362307cd12
766c8be2-dc5c-4e3a-b88d-a14ce6143492
8b3b9c76-4d5f-4800-a297-9aa6ae23da1b
884e891e-2454-48a8-9cfb-aa90a8206b88
98665da2-614a-41bb-95d2-d7adc55ff5df
898b809b-2b46-443d-babb-c374ce638b94
ad7a3544-2c1f-4ffe-beca-ad373cedafaf
06de2a1d-e72f-4d10-8cb4-34286e4a859e
fe45458d-e425-4e65-85b6-1890bcb7ad70

In [22]:
# Org-Site Comb
df_oc_stat.filter(F.col("site_id").isin(sites_check & old_site_list)).select("site_id", "org_id").distinct().show(100, truncate=False
                                                                                                               )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+------------------------------------+
|site_id                             |org_id                              |
+------------------------------------+------------------------------------+
|898b809b-2b46-443d-babb-c374ce638b94|8fd6d6f0-1fb3-40de-b980-1f95ba9aaa24|
|9c103626-af9d-41f4-ac8c-254cb61863f6|cabf8fbf-3567-4d73-bab3-8648f3e981c7|
|f8a3c86b-6472-488e-a6c9-ee42d70a6f44|92c9ecec-3bc8-4ff6-9317-e649aab1a74c|
|884e891e-2454-48a8-9cfb-aa90a8206b88|6acb14a0-29bb-43d8-b211-032539551cb0|
|7cf7111a-b9e9-48df-86ef-230fae9c5a58|5fcef287-c760-412e-8500-c6f1c825923f|
|d08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8|bd71930f-36c8-4836-851e-825dd4998e29|
|03baf34e-0186-4037-b8dc-88c80649532d|79ad4827-0600-4de2-9ef5-5a47ec5dc0aa|
|06de2a1d-e72f-4d10-8cb4-34286e4a859e|85837208-b9e3-4b81-a9be-daa50ddf2b5f|
|98665da2-614a-41bb-95d2-d7adc55ff5df|758a1f7f-6d65-4d85-90b2-0775b8b476bc|
|2db0c421-8401-43bf-9a5c-0d901628a5e2|cd5d9339-d5dc-4a8c-a733-c90ff9c8e893|
|ad7a3544-2c

In [10]:
def compute_v2(edges, nodes, df_oc, df_new_uplink):

    df_oc = consistency_check_v2(df_oc, df_new_uplink).persist()

    old_preds_switches = df_oc.filter(F.size("uplink_combined")>0).count()
    any_match = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)).count()
    total_match = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")==0)&(F.col("pred_mismatch")==0)).count()
    less_uplink_pred = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")>0)&(F.col("pred_mismatch")==0)).count()
    more_uplink_pred = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")==0)&(F.col("pred_mismatch")>0)).count()
    new_uplink_pred = df_oc.filter((~F.col("uplink_new").isNull()) & (F.size("uplink_combined")==0)).count()

    no_uplink_pred = df_oc.filter((F.col("uplink_new").isNull()) & (F.size("uplink_combined")>0)).count()
    diff_uplink_pred = df_oc.filter((~F.col("uplink_new").isNull()) & (F.size("uplink_combined")>0) & (F.col("match")==0)).count()
    
    # Logging Consistency Metrics
    print('Total switches with existing predictions : {}'.format(old_preds_switches))

    print('Among existing, any match : {}'.format(any_match))

    print('Among existing, total match : {}'.format(total_match))

    print('Among existing, less uplinks predicted : {}'.format(less_uplink_pred))

    print('Among existing, more uplinks predicted : {}'.format(more_uplink_pred))

    print('New uplinks predicted : {}'.format(new_uplink_pred))
    
    print('Among existing, no uplink predicted: {}'.format(no_uplink_pred))
    
    print('Among existing, different uplink predicted: {}'.format(diff_uplink_pred))

    # Importing from old predictions where new predictions are unavailable
    updated_edges = mix_predictions(df_oc, edges).persist()

    active_edges, candidate_node = process_data(updated_edges, nodes)

    active_edges_count = active_edges.count()
    print('Total active_edges_count : {}'.format(active_edges_count))

    site_graph_cycle_true = get_sites_with_cycles(active_edges).persist()

    sites_with_cycles = site_graph_cycle_true.count()
    print('Sites with cycles in device collection(relType==uplink) = {}'.format(sites_with_cycles))

    # For site coverage
    df_cycle_cov = site_graph_cycle_true.join(candidate_node, 
                                    (site_graph_cycle_true.siteId == candidate_node.siteId_copy), 
                                    how="left").drop("siteId_copy")
    df_cov_stats = df_cycle_cov.withColumn("nodes_covered", 
                    udf_get_path(F.struct("graph", "mac_list"))) \
        .withColumn("visited_count", udf_visit_count(F.col("nodes_covered"))) \
        .withColumn("node_count", udf_node_count(F.col("nodes_covered"))) \
        .withColumn("coverage", udf_coverage(F.struct(F.col("visited_count"), F.col("node_count")))) \
        .select(*["siteId", "visited_count", "node_count", "coverage"]).persist()

    top_loop_count = df_cov_stats.filter(F.col("visited_count") == 0).count()
    print('Total sites with top level loops {}'.format(top_loop_count))
    
    stats = df_cov_stats.agg({"visited_count":"sum", "node_count": "sum", "coverage":"mean"}).collect()
    print('Coverage in sites with loops {}'.format(stats[0][0]))

    print('Devices left out in sites with loops {}'.format(stats[0][2] - stats[0][1]))
    
    df_site_graph_all_cycles = site_graph_cycle_true.withColumn('cycles', udf_find_cycles(F.col('graph')))
    # df_site_graph_all_cycles_nx = site_graph_cycle_true.withColumn('cycles', udf_find_cycles_nx(F.col('graph')))
    # df_graph_cycles_nx_pd = df_site_graph_all_cycles_nx.select('siteId', 'cycles').toPandas()

    df_graph_cycles = df_site_graph_all_cycles \
        .select('siteId', 'cycles'). \
        withColumn('cycle_length', udf_array_size(F.col('cycles'))). \
        withColumn('num_cycles', F.size(F.col('cycle_length')))

    total_cycles = df_graph_cycles.select(
        F.sum(F.col('num_cycles')).alias('total_cycles')
    ).rdd.map(lambda x: x.asDict()).first()
    if total_cycles:
        print('new_model_total_cycles: ', total_cycles)

    df_device_macs = df_site_graph_all_cycles.select('siteId', 'cycles') \
        .withColumn('macs', udf_extract_mac(F.col('cycles')))
    df_macs = df_device_macs.select(F.explode(F.col('macs')).alias('mac')).distinct()

    total_device_count = df_macs.count()
    print('Total device involved in loops {}'.format(total_device_count))

    df_combined_nodes = df_macs.join(nodes, df_macs.mac == nodes.mac)

    device_types = df_combined_nodes.groupBy('deviceType') \
        .count().rdd.map(lambda x: x.asDict()).collect()
    device_types = {r.get('deviceType'): r.get('count') for r in device_types}
    if device_types:
        print('Total device_types {}'.format(device_types))
    
    return df_cov_stats

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def consistency_check_v2(df_oc, df_new):
    df_new = df_new.filter(F.col("uplink_value")==1).groupby("site_id", "switch_id").agg(
        F.collect_list("name").alias("uplink_new"),
        F.max("n_ind").alias("n_ind")
    )

    rank_window = Window().partitionBy(F.col('switch_id')).orderBy(F.col("when").desc())
    df_oc = df_oc.withColumn("rank", F.rank().over(rank_window)).filter(F.col("rank")==1)
    df_oc = df_oc.select("site_id", "switch_id", "uplink_heuristic", "uplink_combined")

    df_oc = df_oc.join(df_new, ["site_id", "switch_id"], how="outer")
    df_oc = df_oc.withColumn("uplink_new", F.when((F.col("n_ind")==0)&(F.size("uplink_heuristic")>0), 
                                                  F.col("uplink_combined")).otherwise(F.col("uplink_new")))

    df_oc = df_oc.withColumn("match", uplink_match("uplink_combined", "uplink_new")) \
        .select('site_id', 'switch_id', 'uplink_combined', 'uplink_new', F.col('match.*'))

    return df_oc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
sites = compute_v2(df_edge, df_node, df_oc_stat, df_uplink)
sites = sites.filter(F.col("visited_count") == 0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total switches with existing predictions : 14488
Among existing, any match : 12255
Among existing, total match : 12010
Among existing, less uplinks predicted : 243
Among existing, more uplinks predicted : 2
New uplinks predicted : 31
Among existing, no uplink predicted: 1209
Among existing, different uplink predicted: 1024
Pre-processing node and edge data
Total active_edges_count : 10904
Building graph in get_sites_with_cycles
Sites with cycles in device collection(relType==uplink) = 67
Total sites with top level loops 54
Coverage in sites with loops 0.08220902797001511
Devices left out in sites with loops 419
new_model_total_cycles:  {'total_cycles': 82}
Total device involved in loops 138
Total device_types {'gateway': 10, 'switch': 128}

In [13]:
site_list = set(
    sites.select('siteId').toPandas()['siteId']
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
site_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'7cf7111a-b9e9-48df-86ef-230fae9c5a58', '0e9177ea-e28f-4741-9595-76ae7b88f755', '09df1500-f1b3-4501-a805-7af21a1fb90f', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', 'a7583402-7806-4970-8b65-fabf0034822e', '4d6abae0-f9eb-40fe-9db2-ccb630d1261b', '785bd2d2-fb41-4c0d-a049-40fe0983a159', '2c355e9f-8cdd-4aeb-811e-26907afb3a46', 'ba5c1279-11bc-49e5-94d4-4a8bd9973a87', '71bb582d-dff6-4569-a281-f2ab3c8fe690', 'd720d950-1ac5-4eba-bf4f-8ae3a4cd74d6', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '88a9ac0f-f166-4a7a-aeff-e3b0f37c0844', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '54f41c7f-42c2-483c-a073-797964bfffed', '9c103626-af9d-41f4-ac8c-254cb61863f6', 'b065a63a-f500-428a-80bf-9bac6301317e', '3604f709-045b-459a-b8df-45a399ab56e4', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'd24911d1-b4ca-488d-8b87-0f1ac512d900', 'd72db198-cb28-48d6-b931-dfe5782e8fe6', '3735ed13-1a9d-4b43-af35-a0a9d829b9cc', '1f43a318-19ce-4917-9fb7-5418e6ab9296',

In [15]:
len(old_site_list), len(site_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(143, 54)

In [16]:
len(old_site_list - site_list), len(site_list - old_site_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(107, 18)

In [17]:
site_list - old_site_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'7cf7111a-b9e9-48df-86ef-230fae9c5a58', '0e9177ea-e28f-4741-9595-76ae7b88f755', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', '10811140-1428-4592-88e5-64ca908a08b2', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

In [ ]:
# date = '2022-05-29'
# hr = '15'
# Count = 21 new sites with loops
# {'0e9177ea-e28f-4741-9595-76ae7b88f755', '10448f99-05ff-423f-b48e-696413d4c019', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '9c103626-af9d-41f4-ac8c-254cb61863f6', 'f2860e64-78af-4708-9b64-a5b1299bf952', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', 'cd70c572-6191-41f7-9d54-885e8121619c', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', '10811140-1428-4592-88e5-64ca908a08b2', 'becdf2ec-cd2a-44e3-b9bb-bba05a2c39e2', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '898b809b-2b46-443d-babb-c374ce638b94', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}

# date = '2022-05-30'
# hr = '03'
# Count = 14 new sites with loops
# {'766c8be2-dc5c-4e3a-b88d-a14ce6143492', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', '898b809b-2b46-443d-babb-c374ce638b94', '06de2a1d-e72f-4d10-8cb4-34286e4a859e', '10811140-1428-4592-88e5-64ca908a08b2', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-30'
# hr = '12'
# Count = 18 new sites with loops
# {'10448f99-05ff-423f-b48e-696413d4c019', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', 'f62e5757-fc31-4442-bf01-eb5f89d27b6c', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', '898b809b-2b46-443d-babb-c374ce638b94', 'e3f9b45c-462c-471c-83be-503534e9f301', '10811140-1428-4592-88e5-64ca908a08b2', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}

# date = '2022-05-30'
# hr = '07'
# Count = 17 new sites with loops
# {'10448f99-05ff-423f-b48e-696413d4c019', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', '99a28ff4-3d06-4762-ae04-9b362307cd12', '2db0c421-8401-43bf-9a5c-0d901628a5e2', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '10811140-1428-4592-88e5-64ca908a08b2', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-25'
# hr = '10'
# Count = 18 new sites with loops
# {'7cf7111a-b9e9-48df-86ef-230fae9c5a58', '0e9177ea-e28f-4741-9595-76ae7b88f755', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', '10811140-1428-4592-88e5-64ca908a08b2', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-27'
# hr = '17'
# Count = 18 new sites with loops
# {'10448f99-05ff-423f-b48e-696413d4c019', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'bd6ca967-f622-40e9-b75a-bbdee6330e15', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '03baf34e-0186-4037-b8dc-88c80649532d', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '884e891e-2454-48a8-9cfb-aa90a8206b88', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '98665da2-614a-41bb-95d2-d7adc55ff5df', '128aa802-6601-4197-bb38-c9c2ae2d3994', '2db0c421-8401-43bf-9a5c-0d901628a5e2', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}



In [ ]:
# date = '2022-05-29'
# hr = '15'
# Count = 25 new sites with loops
# {'04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '8e443f2e-914e-43fc-91e6-0d732c82d610', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '9c103626-af9d-41f4-ac8c-254cb61863f6', 'f2860e64-78af-4708-9b64-a5b1299bf952', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', 'cd70c572-6191-41f7-9d54-885e8121619c', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'f7cb3d7a-c1fb-4717-9ecd-b81190342f17', '10811140-1428-4592-88e5-64ca908a08b2', 'becdf2ec-cd2a-44e3-b9bb-bba05a2c39e2', '61236b85-d4d6-4289-b093-3ff48a9adb65', '1645f666-d2c8-4574-a300-b52d49ce4167', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '76d6f4b2-ce21-4b80-bfef-b2cc06d8b4ca', '898b809b-2b46-443d-babb-c374ce638b94', '073e3072-d1fe-4560-8d1a-99b79bd3ed18', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}

# date = '2022-05-30'
# hr = '03'
# Count = 20 new sites with loops
# {'04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '8e443f2e-914e-43fc-91e6-0d732c82d610', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '128aa802-6601-4197-bb38-c9c2ae2d3994', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'f7cb3d7a-c1fb-4717-9ecd-b81190342f17', '2b5b0e11-5f54-4d12-a2a2-7a7a4cd1a5f8', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', '10811140-1428-4592-88e5-64ca908a08b2', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '61236b85-d4d6-4289-b093-3ff48a9adb65', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '1645f666-d2c8-4574-a300-b52d49ce4167', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '898b809b-2b46-443d-babb-c374ce638b94', '76d6f4b2-ce21-4b80-bfef-b2cc06d8b4ca', '06de2a1d-e72f-4d10-8cb4-34286e4a859e', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-30'
# hr = '12'
# Count = 23 new sites with loops
# {'04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '8e443f2e-914e-43fc-91e6-0d732c82d610', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '9c103626-af9d-41f4-ac8c-254cb61863f6', '898b809b-2b46-443d-babb-c374ce638b94', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'e3f9b45c-462c-471c-83be-503534e9f301', 'f62e5757-fc31-4442-bf01-eb5f89d27b6c', '8bb1508b-d45b-4282-b1c5-fc5916332295', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'f7cb3d7a-c1fb-4717-9ecd-b81190342f17', '10811140-1428-4592-88e5-64ca908a08b2', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '1645f666-d2c8-4574-a300-b52d49ce4167', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '76d6f4b2-ce21-4b80-bfef-b2cc06d8b4ca', 'e4c07082-683b-4c4e-8e4c-336f8d5eb215', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}

# date = '2022-05-30'
# hr = '07'
# Count = 23 new sites with loops
# {'04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '8e443f2e-914e-43fc-91e6-0d732c82d610', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '9c103626-af9d-41f4-ac8c-254cb61863f6', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '8bb1508b-d45b-4282-b1c5-fc5916332295', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'f7cb3d7a-c1fb-4717-9ecd-b81190342f17', '2b5b0e11-5f54-4d12-a2a2-7a7a4cd1a5f8', '10811140-1428-4592-88e5-64ca908a08b2', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '61236b85-d4d6-4289-b093-3ff48a9adb65', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '1645f666-d2c8-4574-a300-b52d49ce4167', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '76d6f4b2-ce21-4b80-bfef-b2cc06d8b4ca', '898b809b-2b46-443d-babb-c374ce638b94', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-25'
# hr = '10'
# Count = 22 new sites with loops
# {'7cf7111a-b9e9-48df-86ef-230fae9c5a58', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', 'c6ca0a19-1344-4b60-8d09-6da9e32ea242', '8e443f2e-914e-43fc-91e6-0d732c82d610', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', '9c103626-af9d-41f4-ac8c-254cb61863f6', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '8bb1508b-d45b-4282-b1c5-fc5916332295', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'f7cb3d7a-c1fb-4717-9ecd-b81190342f17', '10811140-1428-4592-88e5-64ca908a08b2', '1645f666-d2c8-4574-a300-b52d49ce4167', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '76d6f4b2-ce21-4b80-bfef-b2cc06d8b4ca', '898b809b-2b46-443d-babb-c374ce638b94', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', 'fe45458d-e425-4e65-85b6-1890bcb7ad70'}

# date = '2022-05-27'
# hr = '17'
# Count = 23 new sites with loops
# {'04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'bd6ca967-f622-40e9-b75a-bbdee6330e15', '8e443f2e-914e-43fc-91e6-0d732c82d610', 'b9a80f71-4d76-457b-b380-4b33aaf5e77a', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '7e86e1cc-e584-43d5-98a8-1f4e718254fb', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '9c103626-af9d-41f4-ac8c-254cb61863f6', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'd08ccc2b-93f7-4d4a-b4dd-12b34eeb1aa8', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '03baf34e-0186-4037-b8dc-88c80649532d', '8bb1508b-d45b-4282-b1c5-fc5916332295', '99a28ff4-3d06-4762-ae04-9b362307cd12', 'f7cb3d7a-c1fb-4717-9ecd-b81190342f17', '61236b85-d4d6-4289-b093-3ff48a9adb65', '8b3b9c76-4d5f-4800-a297-9aa6ae23da1b', '1645f666-d2c8-4574-a300-b52d49ce4167', '884e891e-2454-48a8-9cfb-aa90a8206b88', '98665da2-614a-41bb-95d2-d7adc55ff5df', '898b809b-2b46-443d-babb-c374ce638b94', 'ad7a3544-2c1f-4ffe-beca-ad373cedafaf', 'ebd01f93-0304-4039-b9dd-fad9b39ec97e'}



In [ ]:
# Total switches with existing predictions : 14226
# Among existing, any match : 11903
# Among existing, total match : 9577
# Among existing, less uplinks predicted : 2322
# Among existing, more uplinks predicted : 4
# New uplinks predicted : 159
# Among existing, no uplink predicted: 928
# Among existing, different uplink predicted: 1395
# Pre-processing node and edge data
# Total active_edges_count : 8445
# Building graph in get_sites_with_cycles
# Sites with cycles in device collection(relType==uplink) = 74
# Total sites with top level loops 58
# Coverage in sites with loops 0.09825742546771024
# Devices left out in sites with loops 413
# new_model_total_cycles:  {'total_cycles': 77}
# Total device involved in loops 150
# Total device_types {'gateway': 9, 'third': 4, 'switch': 141}

# 30-May hr =03
# Total switches with existing predictions : 14551
# Among existing, any match : 11850
# Among existing, total match : 11603
# Among existing, less uplinks predicted : 246
# Among existing, more uplinks predicted : 1
# New uplinks predicted : 23
# Among existing, no uplink predicted: 1648
# Among existing, different uplink predicted: 1053
# Pre-processing node and edge data
# Total active_edges_count : 10935
# Building graph in get_sites_with_cycles
# Sites with cycles in device collection(relType==uplink) = 69
# Total sites with top level loops 51
# Coverage in sites with loops 0.10557038390981978
# Devices left out in sites with loops 452
# new_model_total_cycles:  {'total_cycles': 82}
# Total device involved in loops 142
# Total device_types {'gateway': 9, 'switch': 133}

# 30-May hr =00
# Total switches with existing predictions : 14576
# Among existing, any match : 12131
# Among existing, total match : 11861
# Among existing, less uplinks predicted : 267
# Among existing, more uplinks predicted : 2
# New uplinks predicted : 20
# Among existing, no uplink predicted: 1350
# Among existing, different uplink predicted: 1095
# Pre-processing node and edge data
# Total active_edges_count : 10899
# Building graph in get_sites_with_cycles
# Sites with cycles in device collection(relType==uplink) = 70
# Total sites with top level loops 53
# Coverage in sites with loops 0.09261849118130548
# Devices left out in sites with loops 458
# new_model_total_cycles:  {'total_cycles': 83}
# Total device involved in loops 144
# Total device_types {'gateway': 10, 'switch': 134}

# 29-May hr =15
# Total switches with existing predictions : 14578
# Among existing, any match : 12178
# Among existing, total match : 11901
# Among existing, less uplinks predicted : 273
# Among existing, more uplinks predicted : 3
# New uplinks predicted : 22
# Among existing, no uplink predicted: 1353
# Among existing, different uplink predicted: 1047
# Pre-processing node and edge data
# Total active_edges_count : 10887
# Building graph in get_sites_with_cycles
# Sites with cycles in device collection(relType==uplink) = 75
# Total sites with top level loops 56
# Coverage in sites with loops 0.10487127562363942
# Devices left out in sites with loops 492
# new_model_total_cycles:  {'total_cycles': 89}
# Total device involved in loops 154
# Total device_types {'gateway': 9, 'switch': 145}

In [9]:
# df_uplink.count(), df_uplink.filter(F.col("uplink_value")==1).count()

# df_new = df_uplink.filter(F.col("uplink_value")==1).groupby("site_id", "switch_id").agg(
#         F.collect_list("name").alias("uplink_new"),
#         F.max("n_ind").alias("n_ind")
#     )

# rank_window = Window().partitionBy(F.col('switch_id')).orderBy(F.col("when").desc())
# df_oc = df_oc_stat.withColumn("rank", F.rank().over(rank_window)).filter(F.col("rank")==1)
# df_oc = df_oc.select("site_id", "switch_id", "uplink_heuristic", "uplink_combined")

# df_oc = df_oc.withColumn("uplink_new", F.when((F.col("n_ind")==0)&(F.size("uplink_heuristic")>0), F.col("uplink_combined")).otherwise(F.col("uplink_new")))

# df_oc.filter((F.col("n_ind")==0)&(F.size("uplink_heuristic")>0)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(323104, 16095)